Use this code to threshold the whole brain images for plotting (for values that would be significant)
You can run two workflows, one thresholds at a given value (set below to 3.09 for t, and 5.44 for F)
for the second workflow, idea is to take the images, take the absolute value and square root (to go from F to t), then create a binarised image keeping any value > 3.09. This output is then used as a mask for the original image.

### settings

In [1]:
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
import nipype.interfaces.fsl.maths as fsl
import pandas as pd
import os, re, json

210701-12:54:38,193 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0


In [2]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
thrsh = pe.Workflow(name="thrshold") # workflow to run the analysis

### data grabbing and sinking

In [3]:
dg = pe.Node(DataGrabber(infields=['sub', 'TR', 'data_folder'], 
                         outfields=['ts']), name='data-grabber')
dg.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dg.inputs.sort_filelist = True
dg.inputs.template='*'
dg.inputs.template_args = {'ts': [['data_folder', 'sub', 'TR']]}
dg.inputs.field_template = {'ts': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/%s/sub-%s/TR%s/FLGLM/spmF*.nii'}

In [4]:
# test it
dg.inputs.sub = '01'
dg.inputs.TR = '700'
dg.inputs.data_folder = 'glmTDFAST_HANDS'
res = dg.run()
res.outputs

210701-12:54:38,918 nipype.workflow INFO:
	 [Node] Setting-up "data-grabber" in "/tmp/tmpmlju9c6f/data-grabber".
210701-12:54:38,924 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
210701-12:54:38,931 nipype.workflow INFO:
	 [Node] Finished "data-grabber".



ts = /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_HANDS/sub-01/TR700/FLGLM/spmF_0001.nii

In [5]:
def printSubPath(fullFilePath):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the container string
    # given the full filepath, this extracts the subject folder and TR strings for input
    # into DataSink
    import os
    import re
    #fname = os.path.normpath(fullFilePath[0])
    fname = os.path.normpath(fullFilePath)
    l = fname.split(os.sep)
    name = [s for s in l if re.search('sub', s)][0], [t for t in l if re.search('TR([0-9])', t)][0]
    name = '/'.join(name)
    name
    return name

In [6]:
printSubPath(res.outputs.ts)

'sub-01/TR700'

In [7]:
# directory requires setting!
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_HANDS"
substitutions = [('_TR_([0-9]*)_sub_([0-9]*)', ''),
                 ('_apply-thrsh([0-9]*)', ''),
                 ('_doabs([0-9]*)', ''),
                 ('_dosqrt([0-9]*)', ''),
                 ('_dobin([0-9]*)', ''),
                 ('_domsk([0-9]*)', '')] 
ds.inputs.regexp_substitutions = substitutions

In [8]:
# to feed into datagrabber
datinf = pe.Node(ni.IdentityInterface(fields=['sub', 'TR']),
                 name='dat-info')
datinf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]
#datinf.iterables = [('sub', ['04']), ('TR', ['700', '1510', '1920'])]
datinf.iterables 

[('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]

## take absolute and square root of images

In [9]:
doabs = pe.MapNode(fsl.UnaryMaths(), iterfield = ['in_file'], name = 'doabs')
doabs.inputs.operation = "abs"

In [10]:
dosqrt = pe.MapNode(fsl.UnaryMaths(), iterfield = ['in_file'], name = 'dosqrt')
dosqrt.inputs.operation = "sqrt"

### threshold

In [11]:
applythrsh = pe.MapNode(fsl.Threshold(), iterfield = ['in_file'], name='apply-thrsh')
applythrsh.inputs.thresh = 5.44
applythrsh.inputs.direction = 'below'
applythrsh.inputs.nan2zeros = True

## binarise

In [12]:
applybin = pe.MapNode(fsl.UnaryMaths(), iterfield = ['in_file'], name = 'dobin')
applybin.inputs.operation = "bin"

## apply binarised image to original image

In [13]:
applymsk = pe.MapNode(fsl.ApplyMask(), iterfield = ['in_file', 'mask_file'], name = 'domsk')

## linking workflow for F contrasts

In [14]:
# thrsh.connect([(datinf, dg, [('sub', 'sub')]),
#                (datinf, dg, [('TR', 'TR')]),
#                (dg, ds, [(('ts', printSubPath), 'container')]),
#                (dg, doabs, [('ts', 'in_file')]),
#                (doabs, dosqrt, [('out_file', 'in_file')]),
#                (dosqrt, applythrsh, [('out_file', 'in_file')]),
#                (applythrsh, applybin, [('out_file', 'in_file')]),
#                (dg, applymsk, [('ts', 'in_file')]),
#                (applybin, applymsk, [('out_file', 'mask_file')]),
#                (applymsk, ds, [('out_file', 'THRSH')])
#               ])
# res = thrsh.run()

### link workflow

In [15]:
thrsh.connect([(datinf, dg, [('sub', 'sub')]),
               (datinf, dg, [('TR', 'TR')]),
               (dg, ds, [(('ts', printSubPath), 'container')]),
               (dg, applythrsh, [('ts', 'in_file')]), 
               (applythrsh, ds, [('out_file', 'THRSH')])
])
res = thrsh.run()

210701-12:54:39,28 nipype.workflow INFO:
	 Workflow thrshold settings: ['check', 'execution', 'logging', 'monitoring']
210701-12:54:39,93 nipype.workflow INFO:
	 Running serially.
210701-12:54:39,94 nipype.workflow INFO:
	 [Node] Setting-up "thrshold.data-grabber" in "/tmp/tmpmlju9c6f/data-grabber".
210701-12:54:39,100 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
210701-12:54:39,105 nipype.workflow INFO:
	 [Node] Finished "thrshold.data-grabber".
210701-12:54:39,106 nipype.workflow INFO:
	 [Node] Setting-up "thrshold.apply-thrsh" in "/tmp/tmpuic8on4t/thrshold/_TR_1920_sub_05/apply-thrsh".
210701-12:54:39,112 nipype.workflow INFO:
	 [Node] Setting-up "_apply-thrsh0" in "/tmp/tmpuic8on4t/thrshold/_TR_1920_sub_05/apply-thrsh/mapflow/_apply-thrsh0".
210701-12:54:39,117 nipype.workflow INFO:
	 [Node] Running "_apply-thrsh0" ("nipype.interfaces.fsl.maths.Threshold"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/der

210701-12:54:40,211 nipype.workflow INFO:
	 [Node] Finished "_apply-thrsh0".
210701-12:54:40,214 nipype.workflow INFO:
	 [Node] Finished "thrshold.apply-thrsh".
210701-12:54:40,215 nipype.workflow INFO:
	 [Node] Setting-up "thrshold.sink-stuff" in "/tmp/tmpitt0q64v/thrshold/_TR_1510_sub_04/sink-stuff".
210701-12:54:40,222 nipype.workflow INFO:
	 [Node] Running "sink-stuff" ("nipype.interfaces.io.DataSink")
210701-12:54:40,223 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_HANDS/sub-04/TR1510/THRSH/_TR_1510_sub_04/_apply-thrsh0/spmF_0001_thresh.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_HANDS/sub-04/TR1510/THRSH///spmF_0001_thresh.nii.gz
210701-12:54:40,228 nipype.workflow INFO:
	 [Node] Finished "thrshold.sink-stuff".
210701-12:54:40,229 nipype.workflow INFO:
	 [Node] Setting-up "thrshold.data-grabber" in "/tmp/tmpmlju9c6f/data-grabber".
210701-12:54:40,233 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io

210701-12:54:41,125 nipype.workflow INFO:
	 [Node] Setting-up "thrshold.apply-thrsh" in "/tmp/tmpx206n3kh/thrshold/_TR_1920_sub_02/apply-thrsh".
210701-12:54:41,131 nipype.workflow INFO:
	 [Node] Setting-up "_apply-thrsh0" in "/tmp/tmpx206n3kh/thrshold/_TR_1920_sub_02/apply-thrsh/mapflow/_apply-thrsh0".
210701-12:54:41,135 nipype.workflow INFO:
	 [Node] Running "_apply-thrsh0" ("nipype.interfaces.fsl.maths.Threshold"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_HANDS/sub-02/TR1920/FLGLM/spmF_0001.nii -nan -thr 5.4400000000 /tmp/tmpx206n3kh/thrshold/_TR_1920_sub_02/apply-thrsh/mapflow/_apply-thrsh0/spmF_0001_thresh.nii.gz
210701-12:54:41,362 nipype.workflow INFO:
	 [Node] Finished "_apply-thrsh0".
210701-12:54:41,365 nipype.workflow INFO:
	 [Node] Finished "thrshold.apply-thrsh".
210701-12:54:41,366 nipype.workflow INFO:
	 [Node] Setting-up "thrshold.sink-stuff" in "/tmp/tmphaf2rnb2/thrshold/_TR_1920_sub_02/sink-stuff".
210701-12:5

210701-12:54:42,283 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_HANDS/sub-01/TR1510/THRSH/_TR_1510_sub_01/_apply-thrsh0/spmF_0001_thresh.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST_HANDS/sub-01/TR1510/THRSH///spmF_0001_thresh.nii.gz
210701-12:54:42,288 nipype.workflow INFO:
	 [Node] Finished "thrshold.sink-stuff".
210701-12:54:42,288 nipype.workflow INFO:
	 [Node] Setting-up "thrshold.data-grabber" in "/tmp/tmpmlju9c6f/data-grabber".
210701-12:54:42,293 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
210701-12:54:42,297 nipype.workflow INFO:
	 [Node] Finished "thrshold.data-grabber".
210701-12:54:42,297 nipype.workflow INFO:
	 [Node] Setting-up "thrshold.apply-thrsh" in "/tmp/tmpoyzz8rag/thrshold/_TR_700_sub_01/apply-thrsh".
210701-12:54:42,303 nipype.workflow INFO:
	 [Node] Setting-up "_apply-thrsh0" in "/tmp/tmpoyzz8rag/thrshold/_TR_700_sub_01/apply-thrsh/mapflow/_apply-thrsh0".
210701-